In [1]:
from pymongo import MongoClient

# Connecting to MongoDB

In [2]:
client = MongoClient("mongodb://localhost:27017")

# Helper Functions

In [3]:
def print_cursor(c):
    for i in cursor:
        print(i)

# Dataset 1

In [4]:
db = client.universities

In [5]:
client.drop_database("universities")

In [6]:
db.universities.insert_many([
{
  "country" : "Spain",
  "city" : "Salamanca",
  "name" : "USAL",
  "location" : {
    "type" : "Point",
    "coordinates" : [ -5.6722512,17, 40.9607792 ]
  },
  "students" : [
    { "year" : 2014, "number" : 24774 },
    { "year" : 2015, "number" : 23166 },
    { "year" : 2016, "number" : 21913 },
    { "year" : 2017, "number" : 21715 }
  ]
},
{
  "country" : "Spain",
  "city" : "Salamanca",
  "name" : "UPSA",
  "location" : {
    "type" : "Point",
    "coordinates" : [ -5.6691191,17, 40.9631732 ]
  },
  "students" : [
    { "year" : 2014, "number" : 4788 },
    { "year" : 2015, "number" : 4821 },
    { "year" : 2016, "number" : 6550 },
    { "year" : 2017, "number" : 6125 }
  ]
}
])


db.courses.insert_many([
{
  "university" : "USAL",
  "name" : "Computer Science",
  "level" : "Excellent"
},
{
  "university" : "USAL",
  "name" : "Electronics",
  "level" : "Intermediate"
},
{
  "university" : "USAL",
  "name" : "Communication",
  "level" : "Excellent"
}
])

# Aggregation stages

## MongoDB `$match`

In [7]:
cursor = db.universities.aggregate([
  { "$match" : { "country" : "Spain", "city" : "Salamanca" } }
])

print_cursor(cursor)

{'_id': ObjectId('6127b3f7876269d2c1724738'), 'country': 'Spain', 'city': 'Salamanca', 'name': 'USAL', 'location': {'type': 'Point', 'coordinates': [-5.6722512, 17, 40.9607792]}, 'students': [{'year': 2014, 'number': 24774}, {'year': 2015, 'number': 23166}, {'year': 2016, 'number': 21913}, {'year': 2017, 'number': 21715}]}
{'_id': ObjectId('6127b3f7876269d2c1724739'), 'country': 'Spain', 'city': 'Salamanca', 'name': 'UPSA', 'location': {'type': 'Point', 'coordinates': [-5.6691191, 17, 40.9631732]}, 'students': [{'year': 2014, 'number': 4788}, {'year': 2015, 'number': 4821}, {'year': 2016, 'number': 6550}, {'year': 2017, 'number': 6125}]}


## MongoDB `$project`

In [8]:
cursor = db.universities.aggregate([
  { "$project" : { "_id" : 0, "country" : 1, "city" : 1, "name" : 1 } }
])

print_cursor(cursor)

{'country': 'Spain', 'city': 'Salamanca', 'name': 'USAL'}
{'country': 'Spain', 'city': 'Salamanca', 'name': 'UPSA'}


## MongoDB `$group`

In [9]:
cursor = db.universities.aggregate([
  { "$group" : { "_id" : "$name", "totaldocs" : { "$sum" : 1 } } }
])

print_cursor(cursor)

{'_id': 'USAL', 'totaldocs': 1}
{'_id': 'UPSA', 'totaldocs': 1}


## MongoDB `$out`

In [10]:
db.universities.aggregate([
  { "$group" : { "_id" : "$name", "totaldocs" : { "$sum" : 1 } } },
  { "$out" : "aggResults" }
])

cursor = db.aggResults.find()
print_cursor(cursor)

{'_id': 'UPSA', 'totaldocs': 1}
{'_id': 'USAL', 'totaldocs': 1}


## MongoDB `$unwind`

In [11]:
cursor = db.universities.aggregate([
  { "$match" : { "name" : "USAL" } },
  { "$unwind" : "$students" }
])

print_cursor(cursor)

{'_id': ObjectId('6127b3f7876269d2c1724738'), 'country': 'Spain', 'city': 'Salamanca', 'name': 'USAL', 'location': {'type': 'Point', 'coordinates': [-5.6722512, 17, 40.9607792]}, 'students': {'year': 2014, 'number': 24774}}
{'_id': ObjectId('6127b3f7876269d2c1724738'), 'country': 'Spain', 'city': 'Salamanca', 'name': 'USAL', 'location': {'type': 'Point', 'coordinates': [-5.6722512, 17, 40.9607792]}, 'students': {'year': 2015, 'number': 23166}}
{'_id': ObjectId('6127b3f7876269d2c1724738'), 'country': 'Spain', 'city': 'Salamanca', 'name': 'USAL', 'location': {'type': 'Point', 'coordinates': [-5.6722512, 17, 40.9607792]}, 'students': {'year': 2016, 'number': 21913}}
{'_id': ObjectId('6127b3f7876269d2c1724738'), 'country': 'Spain', 'city': 'Salamanca', 'name': 'USAL', 'location': {'type': 'Point', 'coordinates': [-5.6722512, 17, 40.9607792]}, 'students': {'year': 2017, 'number': 21715}}


## MongoDB `$sort`

In [12]:
cursor = db.universities.aggregate([
  { "$match" : { "name" : "USAL" } },
  { "$unwind" : "$students" },
  { "$project" : { "_id" : 0, "students.year" : 1, "students.number" : 1 } },
  { "$sort" : { "students.number" : -1 } }
])

print_cursor(cursor)

{'students': {'year': 2014, 'number': 24774}}
{'students': {'year': 2015, 'number': 23166}}
{'students': {'year': 2016, 'number': 21913}}
{'students': {'year': 2017, 'number': 21715}}


## MongoDB `$limit`

In [13]:
cursor = db.universities.aggregate([
  { "$match" : { "name" : "USAL" } },
  { "$unwind" : "$students" },
  { "$project" : { "_id" : 0, "students.year" : 1, "students.number" : 1 } },
  { "$sort" : { "students.number" : -1 } },
  { "$limit" : 2 }
])

print_cursor(cursor)

{'students': {'year': 2014, 'number': 24774}}
{'students': {'year': 2015, 'number': 23166}}


## MongoDB `$addFields`

In [14]:
cursor = db.universities.aggregate([
  { "$match" : { "name" : "USAL" } },
  { "$addFields" : { "foundation_year" : 1218 } }
])

print_cursor(cursor)

{'_id': ObjectId('6127b3f7876269d2c1724738'), 'country': 'Spain', 'city': 'Salamanca', 'name': 'USAL', 'location': {'type': 'Point', 'coordinates': [-5.6722512, 17, 40.9607792]}, 'students': [{'year': 2014, 'number': 24774}, {'year': 2015, 'number': 23166}, {'year': 2016, 'number': 21913}, {'year': 2017, 'number': 21715}], 'foundation_year': 1218}


## MongoDB `$count`

In [15]:
cursor = db.universities.aggregate([
  { "$unwind" : "$students" },
  { "$count" : "total_documents" }
])

print_cursor(cursor)

{'total_documents': 8}


## MongoDB `$lookup`

If you want this query to run fast, you are going to need to index the `name` field in the `universities` collection and the `university` field in the `courses` collection.

In [16]:
cursor = db.universities.aggregate([
  { "$match" : { "name" : "USAL" } },
  { "$project" : { "_id" : 0, "name" : 1 } },
  { "$lookup" : {
      "from" : "courses",
      "localField" : "name",
      "foreignField" : "university",
      "as" : "courses"
  } }
])

print_cursor(cursor)

{'name': 'USAL', 'courses': [{'_id': ObjectId('6127b3f7876269d2c172473a'), 'university': 'USAL', 'name': 'Computer Science', 'level': 'Excellent'}, {'_id': ObjectId('6127b3f7876269d2c172473b'), 'university': 'USAL', 'name': 'Electronics', 'level': 'Intermediate'}, {'_id': ObjectId('6127b3f7876269d2c172473c'), 'university': 'USAL', 'name': 'Communication', 'level': 'Excellent'}]}


## MongoDB `$sortByCount`

This stage is a shortcut for grouping, counting and then sorting in descending order the number of different values in a field.

In [17]:
cursor = db.courses.aggregate([
  { "$sortByCount" : "$level" }
])

print_cursor(cursor)

{'_id': 'Excellent', 'count': 2}
{'_id': 'Intermediate', 'count': 1}


## MongoDB `$facet`

Sometimes when creating a report on data, you find that you need to do the same preliminary processing for a number of reports, and you are faced with having to create and maintain an intermediate collection.

In [18]:
cursor = db.universities.aggregate([
  { "$match" : { "name" : "USAL" } },
  { "$lookup" : {
      "from" : "courses",
      "localField" : "name",
      "foreignField" : "university",
      "as" : "courses"
  } }
])

print_cursor(cursor)

{'_id': ObjectId('6127b3f7876269d2c1724738'), 'country': 'Spain', 'city': 'Salamanca', 'name': 'USAL', 'location': {'type': 'Point', 'coordinates': [-5.6722512, 17, 40.9607792]}, 'students': [{'year': 2014, 'number': 24774}, {'year': 2015, 'number': 23166}, {'year': 2016, 'number': 21913}, {'year': 2017, 'number': 21715}], 'courses': [{'_id': ObjectId('6127b3f7876269d2c172473a'), 'university': 'USAL', 'name': 'Computer Science', 'level': 'Excellent'}, {'_id': ObjectId('6127b3f7876269d2c172473b'), 'university': 'USAL', 'name': 'Electronics', 'level': 'Intermediate'}, {'_id': ObjectId('6127b3f7876269d2c172473c'), 'university': 'USAL', 'name': 'Communication', 'level': 'Excellent'}]}


In [19]:
cursor = db.universities.aggregate([
  { "$match" : { "name" : "USAL" } },
  { "$lookup" : {
      "from" : "courses",
      "localField" : "name",
      "foreignField" : "university",
      "as" : "courses"
  } },
         { "$unwind" : "$courses" },
         { "$sortByCount" : "$courses.level" }
])

print_cursor(cursor)

{'_id': 'Excellent', 'count': 2}
{'_id': 'Intermediate', 'count': 1}


In [20]:
cursor = db.universities.aggregate([
  { "$match" : { "name" : "USAL" } },
  { "$lookup" : {
      "from" : "courses",
      "localField" : "name",
      "foreignField" : "university",
      "as" : "courses"
  } },
         { "$unwind" : "$students" },
         { "$project" : { "_id" : 0, "students" : 1 } },
         { "$sort" : { "students.number" : 1 } },
         { "$limit" : 1 }
])

print_cursor(cursor)

{'students': {'year': 2017, 'number': 21715}}


In [21]:
cursor = db.universities.aggregate([
  { "$match" : { "name" : "USAL" } },
  { "$lookup" : {
      "from" : "courses",
      "localField" : "name",
      "foreignField" : "university",
      "as" : "courses"
  } },
  { "$facet" : {
      "countingLevels" :
      [
         { "$unwind" : "$courses" },
         { "$sortByCount" : "$courses.level" }
      ],
      "yearWithLessStudents" :
      [
         { "$unwind" : "$students" },
         { "$project" : { "_id" : 0, "students" : 1 } },
         { "$sort" : { "students.number" : 1 } },
         { "$limit" : 1 }
      ]
  } }
])

print_cursor(cursor)

{'countingLevels': [{'_id': 'Excellent', 'count': 2}, {'_id': 'Intermediate', 'count': 1}], 'yearWithLessStudents': [{'students': {'year': 2017, 'number': 21715}}]}


# Dataset 2

Use `mongoimport` to load http://media.mongodb.org/zips.json into your mongod instance.


In [22]:
db = client.zips

In [23]:
cursor = db.zipcodes.find({"_id" : "47906"})
print_cursor(cursor)

{'_id': '47906', 'city': 'WEST LAFAYETTE', 'loc': [-86.923661, 40.444025], 'pop': 54702, 'state': 'IN'}


# Aggregation Pipeline

### Return States with Populations above 10 Million

`SELECT state, SUM(pop) AS totalPop
FROM zipcodes
GROUP BY state
HAVING totalPop >= (10*1000*1000)`

In [24]:
cursor = db.zipcodes.aggregate( [
   { "$group": { "_id": "$state", "totalPop": { "$sum": "$pop" } } },
   { "$match": { "totalPop": { "$gte": 10*1000*1000 } } }
] )

print_cursor(cursor)

{'_id': 'TX', 'totalPop': 16984601}
{'_id': 'CA', 'totalPop': 29754890}
{'_id': 'NY', 'totalPop': 17990402}
{'_id': 'OH', 'totalPop': 10846517}
{'_id': 'IL', 'totalPop': 11427576}
{'_id': 'FL', 'totalPop': 12686644}
{'_id': 'PA', 'totalPop': 11881643}


### Return Average City Population by State

In [25]:
cursor = db.zipcodes.aggregate( [
   { "$group": { "_id": { "state": "$state", "city": "$city" }, "pop": { "$sum": "$pop" } } },
   { "$group": { "_id": "$_id.state", "avgCityPop": { "$avg": "$pop" } } }
] )

print_cursor(cursor)

{'_id': 'PA', 'avgCityPop': 8679.067202337472}
{'_id': 'OK', 'avgCityPop': 6155.743639921722}
{'_id': 'MS', 'avgCityPop': 7524.023391812865}
{'_id': 'VA', 'avgCityPop': 8526.177931034483}
{'_id': 'NC', 'avgCityPop': 10622.815705128205}
{'_id': 'LA', 'avgCityPop': 10465.496277915632}
{'_id': 'SD', 'avgCityPop': 1839.6746031746031}
{'_id': 'OR', 'avgCityPop': 8262.561046511628}
{'_id': 'NM', 'avgCityPop': 5872.360465116279}
{'_id': 'AK', 'avgCityPop': 2976.4918032786886}
{'_id': 'TN', 'avgCityPop': 9656.350495049504}
{'_id': 'VT', 'avgCityPop': 2315.8765432098767}
{'_id': 'DE', 'avgCityPop': 14481.91304347826}
{'_id': 'NV', 'avgCityPop': 18209.590909090908}
{'_id': 'TX', 'avgCityPop': 13775.02108678021}
{'_id': 'WV', 'avgCityPop': 2771.4775888717154}
{'_id': 'MI', 'avgCityPop': 12087.512353706112}
{'_id': 'CA', 'avgCityPop': 27756.42723880597}
{'_id': 'NJ', 'avgCityPop': 15775.89387755102}
{'_id': 'DC', 'avgCityPop': 303450.0}
{'_id': 'ID', 'avgCityPop': 4320.811158798283}
{'_id': 'IN', 

### Return Largest and Smallest Cities by State

In [26]:
cursor = db.zipcodes.aggregate( [
   { "$group":
      {
        "_id": { "state": "$state", "city": "$city" },
        "pop": { "$sum": "$pop" }
      }
   },
   { "$sort": { "pop": 1 } },
   { "$group":
      {
        "_id" : "$_id.state",
        "biggestCity":  { "$last": "$_id.city" },
        "biggestPop":   { "$last": "$pop" },
        "smallestCity": { "$first": "$_id.city" },
        "smallestPop":  { "$first": "$pop" }
      }
   },
  { "$project":
    { "_id": 0,
      "state": "$_id",
      "biggestCity":  { "name": "$biggestCity",  "pop": "$biggestPop" },
      "smallestCity": { "name": "$smallestCity", "pop": "$smallestPop" }
    }
  }
] )

print_cursor(cursor)

{'biggestCity': {'name': 'PORTLAND', 'pop': 518543}, 'smallestCity': {'name': 'LYONS', 'pop': 0}, 'state': 'OR'}
{'biggestCity': {'name': 'MIAMI', 'pop': 825232}, 'smallestCity': {'name': 'CECIL FIELD NAS', 'pop': 0}, 'state': 'FL'}
{'biggestCity': {'name': 'SIOUX FALLS', 'pop': 102046}, 'smallestCity': {'name': 'ZEONA', 'pop': 8}, 'state': 'SD'}
{'biggestCity': {'name': 'NEW ORLEANS', 'pop': 496937}, 'smallestCity': {'name': 'FORDOCHE', 'pop': 0}, 'state': 'LA'}
{'biggestCity': {'name': 'VIRGINIA BEACH', 'pop': 385080}, 'smallestCity': {'name': 'WALLOPS ISLAND', 'pop': 0}, 'state': 'VA'}
{'biggestCity': {'name': 'ALBUQUERQUE', 'pop': 449584}, 'smallestCity': {'name': 'REGINA', 'pop': 0}, 'state': 'NM'}
{'biggestCity': {'name': 'CHARLOTTE', 'pop': 465833}, 'smallestCity': {'name': 'GLOUCESTER', 'pop': 0}, 'state': 'NC'}
{'biggestCity': {'name': 'BURLINGTON', 'pop': 39127}, 'smallestCity': {'name': 'UNIV OF VERMONT', 'pop': 0}, 'state': 'VT'}
{'biggestCity': {'name': 'LAS VEGAS', 'pop':